<a href="https://colab.research.google.com/github/Hydenx2004/RAG-using-Qdrant/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# sUPTOOSRgIYQ9lHspNw08UrNTp1EgFKxkPmoIHEkjS7Tv_38QztOGw
#https://246925e6-242e-4987-b0cb-ec41da426346.europe-west3-0.gcp.cloud.qdrant.io:6333

In [1]:
!pip install langchain qdrant_client openai tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 397.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.

In [2]:
!pip install -U langchain-community # install the langchain-community package


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.1 MB/s eta 0:00:00


##IMPORTING LIBRARIES


In [6]:
from langchain.vectorstores import Qdrant
from langchain.embeddings.openai import OpenAIEmbeddings
import qdrant_client
import os

###CREATING CLIENT

In [7]:
os.environ['QDRANT_HOST'] = "https://246925e6-242e-4987-b0cb-ec41da426346.europe-west3-0.gcp.cloud.qdrant.io:6333"
os.environ['QDRANT_API_KEY'] = "sUPTOOSRgIYQ9lHspNw08UrNTp1EgFKxkPmoIHEkjS7Tv_38QztOGw"


client = qdrant_client.QdrantClient(
        os.getenv("QDRANT_HOST"),
        api_key=os.getenv("QDRANT_API_KEY")
    )

### CREATING COLLECTION

In [37]:
os.environ['QDRANT_COLLECTION'] = "my_collection"

collection_config = qdrant_client.http.models.VectorParams(
        size=384,
        distance=qdrant_client.http.models.Distance.COSINE
    )

client.recreate_collection(
    collection_name=os.getenv("QDRANT_COLLECTION"),
    vectors_config=collection_config
)

<ipython-input-37-942d100afc80>:8: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


True

###CREATING THE VECTOR STORE

In [31]:
!pip install sentence_transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [32]:
from sentence_transformers import SentenceTransformer


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [38]:
model = SentenceTransformer('all-MiniLM-L6-v2')

def embed_texts(texts):
    return model.encode(texts, convert_to_tensor=True).tolist()

vectorstore = Qdrant(
        client=client,
        collection_name=os.getenv("QDRANT_COLLECTION"),
        embeddings=embed_texts
    )

###ADDING DOCUMENTS TO VECTOR DATABASE

In [75]:
from langchain.text_splitter import CharacterTextSplitter

def get_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

with open("story.txt") as f:
    raw_text = f.read()

texts = get_chunks(raw_text)

In [76]:
texts

['The Chronicles of Aetheria: The Lost Kingdom\nChapter 1: The Awakening\nIn the mystical land of Aetheria, where floating islands drifted in a sky of perpetual twilight, a grand kingdom once thrived. The Kingdom of Aetheria was known for its majestic towers, glowing flora, and the powerful Aether Crystal that maintained the balance of magic and reality. However, the kingdom mysteriously vanished from the annals of history, leaving only myths and legends.\nOne fateful night, a young scholar named Elara, who lived in the humble village of Lumoria, discovered an ancient tome buried deep in the archives of her library. The tome spoke of a prophecy regarding the return of Aetheria and a chosen one who would restore its glory. The prophecy hinted at a hidden artifact, the Celestial Amulet, which could reveal the kingdom’s location.\nChapter 2: The Quest Begins',
 'Chapter 2: The Quest Begins\nElara, driven by curiosity and a sense of destiny, embarked on her journey to find the Celestial Am

In [77]:
vectorstore.add_texts(texts)

['8a5ecfb2496d480097eb2c07513518fd',
 'fe8d250f8e8d42529ba3714ac14db632',
 '10e441d5ed1f4b74bae96a31f9c3b216',
 '1ca682d994e649feb87efba9a0534ceb',
 'f620b03bf7db443b9cc9ea30169dfc2f']

In [42]:
!pip install transformers

In [67]:
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer,pipeline

In [68]:
model_name = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_name)
local_model = AutoModelForCausalLM.from_pretrained(model_name)
pipeline = pipeline("text-generation", model=local_model, tokenizer=tokenizer,max_length=1000,pad_token_id=tokenizer.eos_token_id,truncation=True)
hf_pipeline = HuggingFacePipeline(pipeline=pipeline)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [90]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(
    llm=hf_pipeline,
    chain_type="stuff",
    retriever = vectorstore.as_retriever(search_kwargs={"k": 1})
)

In [93]:
query = "Who is the protagonist of the story?"
response = qa.run(query)
print(response)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

The Chronicles of Eldoria: The Quest for the Crystal of Eternity
Chapter 1: The Prophecy
In the ancient land of Eldoria, where magic and mysticism intertwined with the very fabric of reality, a prophecy was foretold. It spoke of a time when darkness would engulf the realm, and only the Crystal of Eternity could restore balance. This crystal, a powerful artifact said to be forged by the gods themselves, was lost to time, hidden in the depths of the Forbidden Mountains.
The prophecy also spoke of a hero, born under the rare celestial event known as the Convergence of the Moons. This hero would possess the courage, wisdom, and strength needed to embark on the perilous quest to retrieve the crystal. Little did anyone know, this hero was a young orphan named Elara, living in the peaceful village of Lyria.
Chapter 2: The Call to A